In [7]:
import pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [2]:
# Load the dataset
glass_df = pd.read_csv('glass.csv')

# Assuming the last column is the target variable
X = glass_df.iloc[:, :-1].values
Y = glass_df.iloc[:, -1].values

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Encode class values as integers and then convert to dummy (one-hot encoded)
encoder = LabelEncoder()
Y_encoded = encoder.fit_transform(Y)
y_dummy = to_categorical(Y_encoded)

In [6]:
glass_df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [3]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_dummy, test_size=0.2, random_state=7)

def create_model():
    # Create model
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(y_dummy.shape[1], activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
# Train an ensemble of models
n_models = 10
models = [create_model() for _ in range(n_models)]
for model in models:
    model.fit(X_train, y_train, epochs=150, verbose=0)


c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Make predictions with each model and average them
predictions = [model.predict(X_test) for model in models]
avg_predictions = np.mean(predictions, axis=0)
ensemble_pred = np.argmax(avg_predictions, axis=1)

# Decode one-hot to single integer label
test_true = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(test_true, ensemble_pred)
print("Ensemble Accuracy: {:.2f}%".format(accuracy * 100))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000214BA809080> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━

In [9]:
# Setup 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=7)
cvscores = []

# Perform 10-fold cross-validation
for train, test in kfold.split(X_normalized, y_dummy):
    # Create a new model for the current fold
    model = create_model()
    
    # Train the model
    model.fit(X_normalized[train], y_dummy[train], epochs=150, verbose=0)
    
    # Evaluate the model
    predictions = model.predict(X_normalized[test])
    y_pred = np.argmax(predictions, axis=1)
    y_true = np.argmax(y_dummy[test], axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    cvscores.append(accuracy)

# Calculate average and standard deviation of the scores
mean_accuracy = np.mean(cvscores)
std_accuracy = np.std(cvscores)
print(f"Mean Accuracy: {mean_accuracy:.2f}%")
print(f"Standard Deviation: {std_accuracy:.2f}%")

c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


c:\Users\ICojocaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Mean Accuracy: 0.61%
Standard Deviation: 0.10%
